In [1]:
#imports
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
#fetch url
url = "https://www.mediaite.com/daily-ratings/cable-news-ratings-monday-june-26-maddow-boosts-msnbc-prime-time-into-first-place/"
base = requests.get(url)
base

<Response [200]>

In [3]:
#convert to html text & test
base_html = base.text 
print(base_html)

<!DOCTYPE html><html lang="en"><head>
<meta charset="utf-8">
<meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=5, minimum-scale=1.0">
<meta name="date" content="2023-06-27T16:37:38+00:00">
<!-- Open Graph -->
<!-- End Open Graph -->
<link rel="shortcut icon" href="https://www.mediaite.com/favicon.ico" /><link rel="apple-touch-icon" href="https://am12.mediaite.com/med/cnt/themes/m2019/images/icon-192x192.png"><link rel="icon" sizes="192x192" href="https://am12.mediaite.com/med/cnt/themes/m2019/images/icon-192x192.png">
		<!-- All in One SEO 4.4.0.1 - aioseo.com -->
		<title>Cable News Ratings Monday June 26: MSNBC, CNN &amp; Fox News</title>
		<meta name="description" content="CNN landed third in total viewers with 770,000. Fox News&#039;s The Five was still the highest-rated show of the day and Fox won in total day viewers and demo viewers." />
		<meta name="robots" content="max-snippet:-1, max-image-preview:large, max-video-preview:-1" />
		<meta name

In [4]:
#make soup object
soup = BeautifulSoup(base_html)
type(soup)

bs4.BeautifulSoup

In [5]:
#selecting just the first of two tables on the page
table = soup.select("table")[0]

In [6]:
#creating header list
first_row = table.select("tbody tr")[0]
header_list = [ cell.text.rstrip("\n") for cell in first_row.select("td") ]
header_list

['ET', 'FNC', 'CNN', 'MSNBC', 'NEWSMAX', 'NEWSNATION']

In [7]:
#selecting non header rows of data from table 
rows_ratings = table.select("tr")[1:19]
len(rows_ratings)
print(rows_ratings)

[<tr>
<td data-sheets-value='{"1":2,"2":"6a"}'>6a</td>
<td data-sheets-value='{"1":2,"2":"FOX AND FRIENDS:\n703"}'>FOX AND FRIENDS:<br/>
703</td>
<td data-sheets-value='{"1":2,"2":"CNN THIS MORNING:\n356"}'>CNN THIS MORNING:<br/>
356</td>
<td data-sheets-value='{"1":2,"2":"MORNING JOE:\n952"}'>MORNING JOE:<br/>
952</td>
<td data-sheets-value='{"1":2,"2":"WENDY BELL COMMON SENSE:\n31"}'>WENDY BELL COMMON SENSE:<br/>
31</td>
<td data-sheets-value='{"1":2,"2":"MORNING IN AMERICA:\n28"}'>MORNING IN AMERICA:<br/>
28</td>
</tr>, <tr>
<td data-sheets-value='{"1":2,"2":"7a"}'>7a</td>
<td data-sheets-value='{"1":2,"2":"FOX AND FRIENDS:\n1024"}'>FOX AND FRIENDS:<br/>
1024</td>
<td data-sheets-value='{"1":2,"2":"CNN THIS MORNING:\n378"}'>CNN THIS MORNING:<br/>
378</td>
<td data-sheets-value='{"1":2,"2":"--"}'>—</td>
<td data-sheets-value='{"1":2,"2":"WAKE UP AMERICA:\n148"}'>WAKE UP AMERICA:<br/>
148</td>
<td data-sheets-value='{"1":2,"2":"--"}'>—</td>
</tr>, <tr>
<td data-sheets-value='{"1":2,"2

In [8]:
#creating list from row data
rating_entries = []
for row in rows_ratings:
    row_cells = []
    for cell in row.select("td"):
        row_cells.append(cell.text.rstrip("\n"))
    rating_entries.append(row_cells)
rating_entries[:3]

[['6a',
  'FOX AND FRIENDS:\n703',
  'CNN THIS MORNING:\n356',
  'MORNING JOE:\n952',
  'WENDY BELL COMMON SENSE:\n31',
  'MORNING IN AMERICA:\n28'],
 ['7a',
  'FOX AND FRIENDS:\n1024',
  'CNN THIS MORNING:\n378',
  '—',
  'WAKE UP AMERICA:\n148',
  '—'],
 ['8a', 'FOX AND FRIENDS:\n1146', 'CNN THIS MORNING:\n478', '—', '—', '—']]

In [9]:
#creatingdataframe
cable26_df = pd.DataFrame(rating_entries, columns=header_list)
cable26_df.head()

,ET,FNC,CNN,MSNBC,NEWSMAX,NEWSNATION
0,6a,FOX AND FRIENDS:\n703,CNN THIS MORNING:\n356,MORNING JOE:\n952,WENDY BELL COMMON SENSE:\n31,MORNING IN AMERICA:\n28
1,7a,FOX AND FRIENDS:\n1024,CNN THIS MORNING:\n378,—,WAKE UP AMERICA:\n148,—
2,8a,FOX AND FRIENDS:\n1146,CNN THIS MORNING:\n478,—,—,—
3,9a,AMERICAS NEWSROOM:\n1198,CNN NEWS CENTRAL:\n595,MORNING JOE:\n1016,NATIONAL REPORT:\n203,—
4,10a,AMERICAS NEWSROOM:\n1266,CNN NEWS CENTRAL:\n572,ANA CABRERA REPORTS:\n842,—,NWSN LIVE W/ MARNI HUGHES:\n37


In [10]:
#dropping everything but fox to create fox column only df
fox26 = cable26_df.drop(columns=['CNN', 'MSNBC','NEWSMAX',"NEWSNATION","ET"])
fox26

,FNC
0,FOX AND FRIENDS:\n703
1,FOX AND FRIENDS:\n1024
2,FOX AND FRIENDS:\n1146
3,AMERICAS NEWSROOM:\n1198
4,AMERICAS NEWSROOM:\n1266
5,"FAULKNER FOCUS, THE:\n1207"
6,OUTNUMBERED:\n1244
7,AMERICA REPORTS:\n1155
8,AMERICA REPORTS:\n1156
9,"STORY, THE:\n1218"


In [11]:
#splitting the string and numbers in the FNC column, while dropping the :\n . And then dropping the original FNC Column
fox26[['Show', '6/26']] = fox26['FNC'].str.split(':\n', expand=True)
fox26 = fox26.drop(columns=['FNC'])
fox26

,Show,6/26
0,FOX AND FRIENDS,703
1,FOX AND FRIENDS,1024
2,FOX AND FRIENDS,1146
3,AMERICAS NEWSROOM,1198
4,AMERICAS NEWSROOM,1266
5,"FAULKNER FOCUS, THE",1207
6,OUTNUMBERED,1244
7,AMERICA REPORTS,1155
8,AMERICA REPORTS,1156
9,"STORY, THE",1218


In [12]:
fox26.to_csv("fox26.csv")